In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import matplotlib.pyplot as plt

In [ ]:
def get_scheduler_stuff(total_timesteps):
    normed_ts = torch.linspace(-1, total_timesteps, total_timesteps+1)/total_timesteps
    s = 0.008
    pre_norm_alpha_cumprods = torch.cos((math.pi/2) * (normed_ts + s) / (1+s)) **2
    alpha_cumprods = pre_norm_alpha_cumprods/ pre_norm_alpha_cumprods[1]
    betas = torch.minimum(1-alpha_cumprods[1:]/alpha_cumprods[:-1], torch.tensor(0.999))

    # Recalculate alpha_cumprods to account for the torch.minimum
    alphas = 1.0-betas
    alpha_cumprods = torch.cumprod(alphas, axis=0)
    alpha_cumprods_prev = torch.cat((torch.tensor([1.0]), alpha_cumprods[:-1]))
    # alpha_cumprod_next = torch.cat((alpha_cumprods[1:], torch.tensor([0.0])))
    posterior_betas = (
                betas * (1.0 - alpha_cumprods_prev) / (1.0 - alpha_cumprods)
            )
    return betas, posterior_betas, alphas, alpha_cumprods, alpha_cumprods_prev

betas, posterior_betas, alphas, alpha_cumprods, alpha_cumprods_prev = get_scheduler_stuff(1000)

sqrt_alpha_cumprods = torch.sqrt(alpha_cumprods)
sqrt_one_minus_alpha_cumprods = torch.sqrt(1.0 - alpha_cumprods)
log_one_minus_alpha_cumprods = torch.log(1.0 - alpha_cumprods)
sqrt_recip_alpha_cumprods = torch.sqrt(1.0 / alpha_cumprods)
sqrt_recipm1_alpha_cumprods = torch.sqrt(1.0 / alpha_cumprods - 1)

# posterior_betas_clipped = torch.cat((([1.0]), posterior_betas[1:]))
log_posterior_betas_clipped = torch.log(torch.cat((posterior_betas[1:2], posterior_betas[1:])))
def get_est_std_dev(model_var_values, t):
    frac = (model_var_values + 1)/2
    return torch.exp((frac*torch.log(betas[t]) - (1-frac)*log_posterior_betas_clipped[t])/2)

mean_est_coef1 = 1/torch.sqrt(alphas)
mean_est_coef2 = -(1-alphas)/(torch.sqrt(alphas)*torch.sqrt(1-alpha_cumprods))
def get_mean_est(x, eps, t):
    return mean_est_coef1[t]*x  + mean_est_coef2[t]*eps

In [ ]:
# How to get mean, variance
# p_mean_variance

# get min_log
# get max_log
# get frac
# get model_variance
# predict the start using self._predict_xstart_from_eps fed into process_xstart
# then apply q_posterior_mean_variance to pred_xstart

def get_pred_xstart(x, t, eps):
    return sqrt_recip_alpha_cumprods[t]*x - sqrt_recipm1_alpha_cumprods[t]*eps

posterior_mean_coef1 = (
    betas * torch.sqrt(alpha_cumprods_prev) / (1.0 - alpha_cumprods)
)
posterior_mean_coef2 = (
    (1.0 - alpha_cumprods_prev) * torch.sqrt(alphas) / (1.0 - alpha_cumprods)
)
def get_posterior_mean(x_start, x_t, t):
    posterior_mean = posterior_mean_coef1[t]*x_start+posterior_mean_coef2[t]*x_t
    return posterior_mean



In [ ]:
nn.Conv2d(10,20,3).weight.shape

In [ ]:
prompt = 'cat'

# Create the text tokens to feed to the model.
tokens = model.tokenizer.encode(prompt)
tokens, mask = model.tokenizer.padded_tokens_and_mask(
    tokens, options['text_ctx']
)

batch_size = 1
tokens=th.tensor([tokens], device='cpu')
mask=th.tensor([mask], dtype=th.bool, device='cpu',)

# model_kwargs = dict(
#     tokens=th.tensor(
#         [tokens] * batch_size, device=device
#     ),
#     mask=th.tensor(
#         [mask] * batch_size,
#         dtype=th.bool,
#         device=device,
#     ),
# )

model.to('cpu')
x = th.randn(1,3,64,64)


for t in list(range(1000))[::-1]:
    print(t)
    with torch.no_grad():
        out = model(x, torch.tensor([t]), tokens=tokens, mask=mask)
        model_eps, model_var_values = out[:,:3], out[:,3:]
        pred_xstart = get_pred_xstart(x=x, eps=model_eps, t=t)
        # mean = get_mean_est(x=x, eps=model_eps, t=t)
        std_dev = get_est_std_dev(model_var_values=model_var_values, t=t)
        mean = get_posterior_mean(x_start=pred_xstart, x_t=x, t=t)
        print('mean:', mean.mean())
        print('std:', std_dev.mean())
        x = mean + torch.randn(*x.shape)*std_dev
        show_images(x)
        print('x:', x.mean())

In [28]:

noise = th.randn(2,3,64,64)
final = None
for sample in diffusion.p_sample_loop_progressive(
    model,
    (2, 3, 64, 64),
    noise=noise,
    cond_fn=None,
    model_kwargs=model_kwargs,
    device=device,
):
    final = sample
    show_images(sample)


TypeError: unsupported operand type(s) for +: 'dict' and 'int'

In [20]:
diffusion.p_sample_loop(
    model_fn,
    (full_batch_size, 3, options["image_size"], options["image_size"]),
    device=device,
    clip_denoised=True,
    progress=True,
    model_kwargs=model_kwargs,
    cond_fn=None,
)[:batch_size]

  6%|▌         | 6/100 [00:42<11:08,  7.12s/it]


KeyboardInterrupt: 

In [ ]:
from PIL import Image
from IPython.display import display
import torch as th

from glide_text2im.download import load_checkpoint
from glide_text2im.model_creation import (
    create_model_and_diffusion,
    model_and_diffusion_defaults,
    model_and_diffusion_defaults_upsampler
)

In [ ]:
has_cuda = torch.cuda.is_available()
device = th.device('cpu' if not has_cuda else 'cuda')

In [ ]:
# Create base model.
has_cuda = False
device = 'cpu'
options = model_and_diffusion_defaults()
options['use_fp16'] = has_cuda
options['timestep_respacing'] = '100' # use 100 diffusion steps for fast sampling
model, diffusion = create_model_and_diffusion(**options)
model.eval()
if has_cuda:
    model.convert_to_fp16()
model.to(device)
model.load_state_dict(load_checkpoint('base', device))
print('total base parameters', sum(x.numel() for x in model.parameters()))

In [ ]:
# Sampling parameters
prompt = "an oil painting of a corgi"
batch_size = 1
guidance_scale = 3.0

# Tune this parameter to control the sharpness of 256x256 images.
# A value of 1.0 is sharper, but sometimes results in grainy artifacts.
upsample_temp = 0.997

In [ ]:
##############################
# Sample from the base model #
##############################

# Create the text tokens to feed to the model.
tokens = model.tokenizer.encode(prompt)
tokens, mask = model.tokenizer.padded_tokens_and_mask(
    tokens, options['text_ctx']
)

# Create the classifier-free guidance tokens (empty)
full_batch_size = batch_size * 2
uncond_tokens, uncond_mask = model.tokenizer.padded_tokens_and_mask(
    [], options['text_ctx']
)

# Pack the tokens together into model kwargs.
model_kwargs = dict(
    tokens=th.tensor(
        [tokens] * batch_size + [uncond_tokens] * batch_size, device=device
    ),
    mask=th.tensor(
        [mask] * batch_size + [uncond_mask] * batch_size,
        dtype=th.bool,
        device=device,
    ),
)

# Create a classifier-free guidance sampling function
def model_fn(x_t, ts, **kwargs):
    half = x_t[: len(x_t) // 2]
    combined = th.cat([half, half], dim=0)
    model_out = model(combined, ts, **kwargs)
    eps, rest = model_out[:, :3], model_out[:, 3:]
    cond_eps, uncond_eps = th.split(eps, len(eps) // 2, dim=0)
    half_eps = uncond_eps + guidance_scale * (cond_eps - uncond_eps)
    eps = th.cat([half_eps, half_eps], dim=0)
    return th.cat([eps, rest], dim=1)

# Sample from the base model.
model.del_cache()
samples = diffusion.p_sample_loop(
    model_fn,
    (full_batch_size, 3, options["image_size"], options["image_size"]),
    device=device,
    clip_denoised=True,
    progress=True,
    model_kwargs=model_kwargs,
    cond_fn=None,
)[:batch_size]
model.del_cache()

# Show the output
show_images(samples)

In [ ]:
def show_images(batch: th.Tensor):
    """ Display a batch of images inline. """
    scaled = ((batch + 1)*127.5).round().clamp(0,255).to(th.uint8).cpu()
    reshaped = scaled.permute(2, 0, 3, 1).reshape([batch.shape[2], -1, 3])
    display(Image.fromarray(reshaped.numpy()))
# show_images(samples)